In [ ]:
import torch
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from deep_beamline_simulation.u_net import ImageProcessing

In [ ]:
"""
File path for train images
training images are the initial intensity of SRX beamline
the output is the output of the SRX beamline
"""
train_file = "../NSLS-II-SRX/rsopt-srw-20220223192627/datasets/results.h5"
output_file = "../image_data/Intensity-At-Sample-63-3m.csv"


In [ ]:
"""
file path for test images
testing images are the initial intensity of CSX
the output is the output of the CSX beamline with varied aperture 
horizontal and vertical is 0.1
"""
test_file = "../image_data/initialInt_262.csv"
test_output_file = "../image_data/sample_555.csv"


In [ ]:
with h5py.File(train_file) as f:
    print(f.keys())
    beam_intensities = f["beamIntensities"]
    ip = ImageProcessing(beam_intensities)
    s = ip.smallest_image_size()
    print(s)

    print(beam_intensities[0])
    
    plt.figure()
    plt.imshow(beam_intensities[0], aspect="auto")
    plt.show()

    normalized_beam_intensities = (beam_intensities - np.mean(beam_intensities)) / np.std(beam_intensities)
    print(normalized_beam_intensities[0])
    
    resized_images = []
    for i in range(normalized_beam_intensities.shape[0]):
        resized_images.append(ip.resize(normalized_beam_intensities[i], height=153, length=351))

    plt.figure()
    plt.imshow(resized_images[0], aspect="auto")
    plt.show()
    
    initial_beam_intensity = pd.read_csv("initial_beam_intensity_srx.csv", skiprows=1).to_numpy()
    normalized_initial_beam_intensity = ( initial_beam_intensity - np.mean(initial_beam_intensity) ) / np.std(initial_beam_intensity)
    resized_initial_beam_intensity = ip.resize(
        normalized_initial_beam_intensity,
        height=153,
        length=351
    )

    with h5py.File("preprocessed_results.h5", mode="w") as preprocessed_results:
        pi = preprocessed_results.create_dataset(
            "preprocessed_initial_beam_intensity",
            (150, 350)
        )
        
        pbi = preprocessed_results.create_dataset(
            "preprocessed_beam_intensities",
            (10, 150, 350)
        )
        for i, resized_image in enumerate(resized_images):
            pbi[i] = resized_image
        
        param_vals = preprocessed_results.create_dataset_like("param_vals", f["paramVals"])
        for i, param_val in enumerate(f["paramVals"]):
            param_vals[i] = param_val

        params = preprocessed_results.create_dataset_like("params", f["params"])
        for i, param in enumerate(f["params"]):
            params[i] = param


In [ ]:
class BeamIntensityDataset:
    def __init__(self, beam_intensities, initial_beam_intensity, params, param_vals):
        self.beam_intensities = beam_intensities
        self.initial_beam_intensity = initial_beam_intensity
        self.params = params
        self.param_vals = param_vals
        
    def __getitem__(self, index):
        return self.beam_intensities[index], self.initial_beam_intensity, self.param_vals[i]
        
    def __len__(self):
        return self.beam_intensities.shape[0]
        